In [62]:
#Convert to .py
!jupytext --to py:percent "C:\Users\ebroh\BetaNYC\School Budgets\School-Budget-and-Allocations\notebooks\streamlitVisuals.ipynb"

zsh:1: command not found: jupytext


In [1]:
import pandas as pd
from re import sub
from decimal import Decimal
#import streamlit as st

def moneyToFloat(money):
    value = Decimal(sub(r'[^\d.]', '', money))
    return value

# Total Allocations vs Budget

In [3]:
district = 5 #Write the district of choice
allocations = pd.read_csv(rf"..\data\district {district}\allocation_district_{district}.csv")
allocations = allocations.drop(columns = ['Unnamed: 0'])
allocations['amountNum'] = allocations['amount'].apply((lambda x: moneyToFloat(x)))
allocations

,location_code,fiscal_year,allocation_category,amount,amountNum
0,M692,2022,AIDP Attendance Shared,"$ 150,490",150490
1,M692,2022,ARPA Academic Recovery,"$ 112,682",112682
2,M692,2022,ARPA Academic Recovery Arts,"$ 28,171",28171
3,M692,2022,ARPA Academic Recovery SE After-School Instruc...,"$ 16,242",16242
4,M692,2022,ARPA Academic Recovery SE Counseling,"$ 2,223",2223
...,...,...,...,...,...
6901,M030,2018,TL Summer in the City Shared,"$ 12,969",12969
6902,M030,2018,TL Translation Services,"$ 1,027",1027
6903,M030,2018,TL Vision for School Improvement,"$ 19,097",19097
6904,M030,2018,Universal PreK Quality Full-Day,"$ 205,776",205776


In [4]:
totalAllocations = allocations.groupby(by=['location_code', 'fiscal_year'])['amountNum'].sum().reset_index()
totalAllocations = totalAllocations.rename(columns = {'amountNum': 'total', 'Type':'type'})
totalAllocations['type'] = 'allocations'
totalAllocations.to_csv(rf"..\data\district {district}\graphs\total_allocations_district_{district}.csv",index=False) #8
totalAllocations.head()

,location_code,fiscal_year,total,type
0,M030,2018,5835002,allocations
1,M030,2019,5817134,allocations
2,M030,2020,5736135,allocations
3,M030,2021,5663435,allocations
4,M030,2022,5490561,allocations


In [6]:
budget = pd.read_csv(f"../data/district {district}/budget_district_{district}.csv")
budget = budget.drop(columns = ['Unnamed: 0'])
budget['amountNum'] = budget['amount'].apply((lambda x: moneyToFloat(x)))
# budget.head()

In [7]:
totalBudget = budget.groupby(by=['location_code', 'fiscal_year'])['amountNum'].sum().reset_index()
totalBudget = totalBudget.rename(columns = {'amountNum': 'total'})
totalBudget['type'] = 'budget'
totalBudget.to_csv(rf"..\data\district {district}\graphs\total_budget_district_{district}.csv",index=False) #7
totalBudget.head()

,location_code,fiscal_year,total,type
0,M030,2018,5830440,budget
1,M030,2019,5702230,budget
2,M030,2020,5677318,budget
3,M030,2021,5586738,budget
4,M030,2022,5490551,budget


In [11]:
totals = pd.concat([totalBudget, totalAllocations])
totals

,location_code,fiscal_year,total,type
0,M030,2018,5830440,budget
1,M030,2019,5702230,budget
2,M030,2020,5677318,budget
3,M030,2021,5586738,budget
4,M030,2022,5490551,budget
...,...,...,...,...
132,M692,2018,4407113,allocations
133,M692,2019,4742274,allocations
134,M692,2020,4737902,allocations
135,M692,2021,4748394,allocations


In [38]:
totalsPivot = totals.pivot(index = ['location_code', 'type'], columns = ['fiscal_year'], values = ['total']).reset_index()
# Flatten the MultiIndex columns
totalsPivot.columns = totalsPivot.columns.to_flat_index()

# Rename the columns
totalsPivot = totalsPivot.reset_index()
new_columns = ['index', 'location_code', 'type', '2018', '2019', '2020', '2021', '2022']
totalsPivot.columns = new_columns
totalsPivot =  totalsPivot.drop(columns = 'index')
totalsPivot.to_csv(rf"..\data\district {district}\graphs\total_budget_and_allocation_district_{district}.csv",index=False) #6 
totalsPivot

,location_code,type,2018,2019,2020,2021,2022
0,M030,allocations,5835002,5817134,5736135,5663435,5490561
1,M030,budget,5830440,5702230,5677318,5586738,5490551
2,M036,allocations,6575446,6774786,6908722,6909623,7355254
3,M036,budget,6480667,6705048,6837152,6853434,7262792
4,M046,allocations,7660059,8358052,8247303,8363241,9354042
5,M046,budget,7620097,8325404,8239311,8357774,9320309
6,M092,allocations,4464084,4647008,4666921,4639564,4538439
7,M092,budget,4445337,4630609,4607173,4587448,4522231
8,M123,allocations,8085767,7481821,6992685,6909528,7888131
9,M123,budget,7904112,7457720,6893315,6822129,7805259


# Funding by Category

In [15]:
allocationCategories = pd.read_csv(r"..\data\allocation_category_lookup.csv")
allocationCategories

,allocation_category,allocation_category_group
0,Contract for Excellence FY 09,Contract for Excellence
1,Contract for Excellence FY 09 HS,Contract for Excellence
2,TL Fair Student Funding,Fair Student Funding
3,TL Fair Student Funding HS,Fair Student Funding
4,TL Funds Over Formula,Fair Student Funding
...,...,...
726,Self Sustaining 14,Other
727,Self Sustaining 35,Other
728,Self Sustaining 12,Other
729,SLP Leg Grant 03,Other


In [16]:
allocationCategories['allocation_category_group'].unique()

array(['Contract for Excellence', 'Fair Student Funding',
       'TL SE Specialized Programs ASD', 'Other',
       'School Support Supplement', 'Preschool', 'Related Services',
       'Title I', 'Title 1'], dtype=object)

In [17]:
district_5_allocation_categories = allocations.merge(allocationCategories, how = 'left', left_on = 'allocation_category', right_on = 'allocation_category')
district_5_allocation_categories

,location_code,fiscal_year,allocation_category,amount,amountNum,allocation_category_group
0,M692,2022,AIDP Attendance Shared,"$ 150,490",150490,Other
1,M692,2022,ARPA Academic Recovery,"$ 112,682",112682,Other
2,M692,2022,ARPA Academic Recovery Arts,"$ 28,171",28171,Other
3,M692,2022,ARPA Academic Recovery SE After-School Instruc...,"$ 16,242",16242,Other
4,M692,2022,ARPA Academic Recovery SE Counseling,"$ 2,223",2223,Other
...,...,...,...,...,...,...
6901,M030,2018,TL Summer in the City Shared,"$ 12,969",12969,Other
6902,M030,2018,TL Translation Services,"$ 1,027",1027,Other
6903,M030,2018,TL Vision for School Improvement,"$ 19,097",19097,Other
6904,M030,2018,Universal PreK Quality Full-Day,"$ 205,776",205776,Preschool


In [18]:
funding_per_category = district_5_allocation_categories.groupby(by = ['location_code', 'fiscal_year', 'allocation_category_group'])['amountNum'].sum().reset_index()
funding_per_category

,location_code,fiscal_year,allocation_category_group,amountNum
0,M030,2018,Contract for Excellence,109432
1,M030,2018,Fair Student Funding,3228153
2,M030,2018,Other,837965
3,M030,2018,Preschool,634955
4,M030,2018,Related Services,518102
...,...,...,...,...
824,M692,2022,Contract for Excellence,45445
825,M692,2022,Fair Student Funding,4348117
826,M692,2022,Other,978364
827,M692,2022,Related Services,84288


In [19]:
funding_per_category_pivot = funding_per_category.pivot(index = ['location_code', 'allocation_category_group'], columns = ['fiscal_year'], values = 'amountNum').reset_index()
funding_per_category_pivot.to_csv(rf"..\data\district {district}\graphs\funding_per_category_district_{district}.csv",index=False) #5
funding_per_category_pivot

fiscal_year,location_code,allocation_category_group,2018,2019,2020,2021,2022
0,M030,Contract for Excellence,109432,106221,104345,104345,104345
1,M030,Fair Student Funding,3228153,2904842,2738880,2487667,2148210
2,M030,Other,837965,942101,724539,865320,1021966
3,M030,Preschool,634955,924557,1132875,1138401,1205471
4,M030,Related Services,518102,460117,512688,509531,457142
...,...,...,...,...,...,...,...
169,M692,Contract for Excellence,52732,48695,45445,45445,45445
170,M692,Fair Student Funding,3847378,3881761,3966819,4052555,4348117
171,M692,Other,436172,736457,679591,576692,978364
172,M692,Related Services,63311,67023,39749,68136,84288


# Enrollment and FSF

In [20]:
enrollment = pd.read_csv(f"..\data\district {district}\district_{district}_demographic_data.csv")
enrollment

,DBN,School Name,Year,Total Enrollment,Grade 3K,Grade PK (Half Day & Full Day),Grade K,Grade 1,Grade 2,Grade 3,...,% White,# Missing Race/Ethnicity Data,% Missing Race/Ethnicity Data,# Students with Disabilities,% Students with Disabilities,# English Language Learners,% English Language Learners,# Poverty,% Poverty,Economic Need Index
0,05M030,P.S. 030 Hernandez/Hughes,2017-18,252,0,16,48,38,38,38,...,2.4%,0,0.0%,75,29.8%,30,11.9%,239,94.8%,Above 95%
1,05M030,P.S. 030 Hernandez/Hughes,2018-19,253,15,15,45,39,32,39,...,3.2%,0,0.0%,60,23.7%,22,8.7%,235,92.9%,93.8%
2,05M030,P.S. 030 Hernandez/Hughes,2019-20,236,17,17,29,31,37,34,...,2.1%,0,0.0%,63,26.7%,16,6.8%,Above 95%,Above 95%,93.2%
3,05M030,P.S. 030 Hernandez/Hughes,2020-21,205,13,22,15,28,25,32,...,2.0%,0,0.0%,57,27.8%,15,7.3%,192,93.7%,92.6%
4,05M030,P.S. 030 Hernandez/Hughes,2021-22,171,9,21,16,22,29,19,...,2.9%,0,0.0%,52,30.4%,18,10.5%,161,94.2%,91.2%
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132,05M692,"High School for Mathematics, Science and Engin...",2017-18,500,0,0,0,0,0,0,...,27.2%,47,9.4%,25,5.0%,1,0.2%,227,45.4%,41.7%
133,05M692,"High School for Mathematics, Science and Engin...",2018-19,491,0,0,0,0,0,0,...,30.3%,57,11.6%,27,5.5%,1,0.2%,204,41.5%,39.3%
134,05M692,"High School for Mathematics, Science and Engin...",2019-20,485,0,0,0,0,0,0,...,29.1%,60,12.4%,28,5.8%,0,0.0%,180,37.1%,37.9%
135,05M692,"High School for Mathematics, Science and Engin...",2020-21,496,0,0,0,0,0,0,...,28.0%,48,9.7%,30,6.0%,0,0.0%,183,36.9%,38.1%


In [21]:
enrollment.columns

Index(['DBN', 'School Name', 'Year', 'Total Enrollment', 'Grade 3K',
       'Grade PK (Half Day & Full Day)', 'Grade K', 'Grade 1', 'Grade 2',
       'Grade 3', 'Grade 4', 'Grade 5', 'Grade 6', 'Grade 7', 'Grade 8',
       'Grade 9', 'Grade 10', 'Grade 11', 'Grade 12', '# Female', '% Female',
       '# Male', '% Male', '# Asian', '% Asian', '# Black', '% Black',
       '# Hispanic', '% Hispanic', '# Multi-Racial', '% Multi-Racial',
       '# Native American', '% Native American', '# White', '% White',
       '# Missing Race/Ethnicity Data', '% Missing Race/Ethnicity Data',
       '# Students with Disabilities', '% Students with Disabilities',
       '# English Language Learners', '% English Language Learners',
       '# Poverty', '% Poverty', 'Economic Need Index'],
      dtype='object')

In [22]:
def removeLeadingZero(string):
    return(sub(r'^\d\d', '', string))

def year2FY(year):
    return(sub(r'-\d\d$', '', year))

In [23]:
enrollment['DBN'] = enrollment['DBN'].apply(lambda x: removeLeadingZero(x))
enrollment['fiscal_year'] = enrollment['Year'].apply(lambda x: year2FY(x))
enrollment

,DBN,School Name,Year,Total Enrollment,Grade 3K,Grade PK (Half Day & Full Day),Grade K,Grade 1,Grade 2,Grade 3,...,# Missing Race/Ethnicity Data,% Missing Race/Ethnicity Data,# Students with Disabilities,% Students with Disabilities,# English Language Learners,% English Language Learners,# Poverty,% Poverty,Economic Need Index,fiscal_year
0,M030,P.S. 030 Hernandez/Hughes,2017-18,252,0,16,48,38,38,38,...,0,0.0%,75,29.8%,30,11.9%,239,94.8%,Above 95%,2017
1,M030,P.S. 030 Hernandez/Hughes,2018-19,253,15,15,45,39,32,39,...,0,0.0%,60,23.7%,22,8.7%,235,92.9%,93.8%,2018
2,M030,P.S. 030 Hernandez/Hughes,2019-20,236,17,17,29,31,37,34,...,0,0.0%,63,26.7%,16,6.8%,Above 95%,Above 95%,93.2%,2019
3,M030,P.S. 030 Hernandez/Hughes,2020-21,205,13,22,15,28,25,32,...,0,0.0%,57,27.8%,15,7.3%,192,93.7%,92.6%,2020
4,M030,P.S. 030 Hernandez/Hughes,2021-22,171,9,21,16,22,29,19,...,0,0.0%,52,30.4%,18,10.5%,161,94.2%,91.2%,2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132,M692,"High School for Mathematics, Science and Engin...",2017-18,500,0,0,0,0,0,0,...,47,9.4%,25,5.0%,1,0.2%,227,45.4%,41.7%,2017
133,M692,"High School for Mathematics, Science and Engin...",2018-19,491,0,0,0,0,0,0,...,57,11.6%,27,5.5%,1,0.2%,204,41.5%,39.3%,2018
134,M692,"High School for Mathematics, Science and Engin...",2019-20,485,0,0,0,0,0,0,...,60,12.4%,28,5.8%,0,0.0%,180,37.1%,37.9%,2019
135,M692,"High School for Mathematics, Science and Engin...",2020-21,496,0,0,0,0,0,0,...,48,9.7%,30,6.0%,0,0.0%,183,36.9%,38.1%,2020


In [24]:
enrollmentFinal = pd.DataFrame()
enrollmentFinal = enrollment[['DBN','fiscal_year', 'Total Enrollment', 'Grade 3K', 'Grade PK (Half Day & Full Day)']]
enrollmentFinal['3K-PreK_enrollment'] = enrollmentFinal['Grade 3K'] + enrollmentFinal['Grade PK (Half Day & Full Day)']
enrollmentFinal['K-5_enrollment'] = enrollmentFinal['Total Enrollment'].str.replace(',','').astype("int64") - enrollmentFinal['3K-PreK_enrollment']
enrollmentFinal = enrollmentFinal.drop(columns = ['Grade 3K', 'Grade PK (Half Day & Full Day)'])
enrollmentFinal

C:\Users\vaish\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\vaish\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,DBN,fiscal_year,Total Enrollment,3K-PreK_enrollment,K-5_enrollment
0,M030,2017,252,16,236
1,M030,2018,253,30,223
2,M030,2019,236,34,202
3,M030,2020,205,35,170
4,M030,2021,171,30,141
...,...,...,...,...,...
132,M692,2017,500,0,500
133,M692,2018,491,0,491
134,M692,2019,485,0,485
135,M692,2020,496,0,496


In [36]:
pivot1 = enrollmentFinal.pivot(index = ['DBN'], columns = ['fiscal_year'], values = '3K-PreK_enrollment').reset_index()
pivot1['Grades'] = '3K-PreK'
pivot2 = enrollmentFinal.pivot(index = ['DBN'], columns = ['fiscal_year'], values = 'K-5_enrollment').reset_index()
pivot2['Grades'] = 'K-5'
enrollmentPivot = pd.concat([pivot1, pivot2])
enrollmentPivot = enrollmentPivot.rename(columns = {'Grades':'Type'})
temp = funding_per_category_pivot[funding_per_category_pivot['allocation_category_group'].isin(['Fair Student Funding', 'Preschool'])]
temp = temp.rename(columns = {'allocation_category_group': 'Type', 'location_code': 'DBN', 2018:'2018', 2019:'2019', 2020:'2020', 2021:'2021'})
temp = temp.drop(columns = [2022])
enrollmentPivot = enrollmentPivot.drop(columns = '2017')
enrollmentPivot = pd.concat([enrollmentPivot, temp])
enrollmentPivot.to_csv(rf"..\data\district {district}\graphs\enrollment_district_{district}.csv",index=False) #4
enrollmentPivot

fiscal_year,DBN,2018,2019,2020,2021,Type
0,M030,30.0,34.0,35.0,30.0,3K-PreK
1,M036,69.0,92.0,42.0,54.0,3K-PreK
2,M046,51.0,44.0,36.0,51.0,3K-PreK
3,M092,18.0,17.0,10.0,15.0,3K-PreK
4,M123,49.0,60.0,40.0,54.0,3K-PreK
...,...,...,...,...,...,...
154,M514,1408207,1469664,1582416,1554330,Fair Student Funding
159,M517,1699861,2018023,2358500,2642484,Fair Student Funding
161,M517,186653,197712,427523,427565,Preschool
165,M670,3475928,3827873,4150704,4326225,Fair Student Funding


# Staff vs Enrollment

In [27]:
teaching_budget = budget[budget['budget_category'].isin(['Paraprofessionals', 'Classroom Teacher', 'Homeroom Teacher', 'Elementary Cluster/Quota', 'Cluster/Quota Teacher'])]
teaching_budget['budget_category'] = teaching_budget['budget_category'].replace(['Classroom Teacher', 'Homeroom Teacher', 'Elementary Cluster/Quota', 'Cluster/Quota Teacher'], 'Teacher')
teaching_budget.sample(10)

C:\Users\vaish\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,location_code,fiscal_year,budget_category,budget_assignment,num_positions,service_type,amount,amountNum
5010,M148,2019,Teacher,SCIENCE - GENERAL SCIENCE,4.0,WHOLE CLASS,"$ 292,290",292290
4174,M161,2019,Teacher,6,1.0,ICT - Special Ed Teacher,"$ 67,294",67294
7222,M036,2019,Teacher,Pre K,2.0,WHOLE CLASS,"$ 277,817",277817
6879,M046,2019,Teacher,3-K,1.0,SELF-CONTAINED SP ED,"$ 115,045",115045
7295,M036,2018,Teacher,1,1.0,ICT - General Ed Teacher,"$ 100,997",100997
3468,M194,2020,Teacher,2,1.0,ICT - General Ed Teacher,"$ 87,763",87763
1746,M362,2020,Teacher,ENGLISH LANGUAGE ARTS,5.0,WHOLE CLASS,"$ 425,910",425910
4106,M161,2020,Teacher,LIBRARY,1.0,SUBJECT SPECIFIC,"$ 89,472",89472
5375,M133,2018,Teacher,PHYSICAL EDUCATION,1.0,WHOLE CLASS,"$ 62,418",62418
5123,M133,2022,Teacher,PHYSICAL EDUCATION,2.0,WHOLE CLASS,"$ 118,042",118042


In [28]:
positions_per_school = teaching_budget.groupby(by = ['location_code', 'fiscal_year', 'budget_category'])['num_positions'].sum().reset_index()
positions_per_school

,location_code,fiscal_year,budget_category,num_positions
0,M030,2018,Paraprofessionals,10.0
1,M030,2018,Teacher,32.0
2,M030,2019,Paraprofessionals,12.0
3,M030,2019,Teacher,31.0
4,M030,2020,Paraprofessionals,12.0
...,...,...,...,...
234,M670,2022,Paraprofessionals,6.0
235,M692,2018,Paraprofessionals,1.0
236,M692,2019,Paraprofessionals,1.0
237,M692,2021,Paraprofessionals,1.0


In [35]:
positions_per_school_pivot = positions_per_school.pivot(index = ['location_code', 'budget_category'], columns = 'fiscal_year', values = 'num_positions').reset_index()
positions_per_school_pivot = positions_per_school_pivot.rename(columns = {'budget_category': 'type'})
positions_per_school_pivot = positions_per_school_pivot.rename(columns = {2018:'2018', 2019:'2019', 2020:'2020', 2021:'2021'})
positions_per_school_pivot.to_csv(rf"..\data\district {district}\graphs\positions_per_school_district_{district}.csv",index=False) #3
positions_per_school_pivot


fiscal_year,location_code,type,2018,2019,2020,2021,2022
0,M030,Paraprofessionals,10.0,12.0,12.0,12.0,11.0
1,M030,Teacher,32.0,31.0,31.0,24.0,22.0
2,M036,Paraprofessionals,20.0,16.0,17.0,18.0,20.0
3,M036,Teacher,35.8,34.0,34.0,33.0,28.0
4,M046,Paraprofessionals,10.0,14.0,19.0,21.0,18.0
5,M046,Teacher,52.0,49.0,48.0,44.0,41.0
6,M092,Paraprofessionals,12.0,11.0,11.0,10.0,7.0
7,M092,Teacher,23.0,21.0,19.0,18.0,15.0
8,M123,Paraprofessionals,7.0,7.0,8.0,12.0,7.0
9,M123,Teacher,41.0,38.0,40.0,38.0,38.0


In [37]:
totalEnrollment = enrollment[['DBN','fiscal_year', 'Total Enrollment']]
totalEnrollment = totalEnrollment.rename(columns = {'DBN': 'location_code'})
totalEnrollmentPivot = totalEnrollment.pivot(index = 'location_code', columns = 'fiscal_year', values = 'Total Enrollment').reset_index()
totalEnrollmentPivot['type'] = 'Total Enrollment'
staffAndEnrollment = pd.concat([totalEnrollmentPivot,positions_per_school_pivot])
staffAndEnrollment = staffAndEnrollment.drop(columns = ['2017', 2022])
staffAndEnrollment.to_csv(rf"..\data\district {district}\graphs\staff_and_enrollment_district_{district}.csv",index=False) #2
staffAndEnrollment.head()

fiscal_year,location_code,2018,2019,2020,2021,type
0,M030,253,236,205,171,Total Enrollment
1,M036,355,374,312,290,Total Enrollment
2,M046,630,557,490,479,Total Enrollment
3,M092,259,243,214,210,Total Enrollment
4,M123,471,461,418,430,Total Enrollment


# Budget Per Staff Member

In [31]:
pos_and_budget = teaching_budget.groupby(by = ['location_code', 'fiscal_year', 'budget_category'])['amountNum'].sum().reset_index()
pos_and_budget

,location_code,fiscal_year,budget_category,amountNum
0,M030,2018,Paraprofessionals,414942
1,M030,2018,Teacher,3083121
2,M030,2019,Paraprofessionals,495617
3,M030,2019,Teacher,2869512
4,M030,2020,Paraprofessionals,543100
...,...,...,...,...
234,M670,2022,Paraprofessionals,154407
235,M692,2018,Paraprofessionals,39317
236,M692,2019,Paraprofessionals,40779
237,M692,2021,Paraprofessionals,39294


In [34]:
budget_per_staff = pos_and_budget.merge(positions_per_school, on = ['location_code','fiscal_year','budget_category'])
budget_per_staff['average_budget'] = budget_per_staff['amountNum'].astype(float)/budget_per_staff['num_positions']
budget_per_staff.to_csv(rf"..\data\district {district}\graphs\budget_per_staff_member_district_{district}.csv",index=False) #1
budget_per_staff

,location_code,fiscal_year,budget_category,amountNum,num_positions,average_budget
0,M030,2018,Paraprofessionals,414942,10.0,41494.200000
1,M030,2018,Teacher,3083121,32.0,96347.531250
2,M030,2019,Paraprofessionals,495617,12.0,41301.416667
3,M030,2019,Teacher,2869512,31.0,92564.903226
4,M030,2020,Paraprofessionals,543100,12.0,45258.333333
...,...,...,...,...,...,...
234,M670,2022,Paraprofessionals,154407,6.0,25734.500000
235,M692,2018,Paraprofessionals,39317,1.0,39317.000000
236,M692,2019,Paraprofessionals,40779,1.0,40779.000000
237,M692,2021,Paraprofessionals,39294,1.0,39294.000000
